<a href="https://colab.research.google.com/github/carlosgarcianter/duende_programador/blob/main/create_typed_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
PROJECT = "dev-bbva-duco-dashboard"
DATASET = "reconciliations"
TABLE_NAME = "[PROCESS_CODE]"

In [ ]:
common_fields = {
  "Id":                   "r.url_id",
  "Process_Code":         "r.process_id",
  "Run_Id":               "CAST(r.run_number AS INTEGER) AS Run_Id",
  "Roll_Up_Relationship": "r.roll_up_relationship",
  "Workflow":             "e.workflow",
  "Status":               "r.status",
  "Group":                "e.group_id",
  "Assigned_To":          "e.assigned_to_user_id",
  "Ageing_Started_At":    "e.creation_timestamp",
  "Age_As_At_Export":     "CAST(e.age AS FLOAT) as Age_As_At_Export",
  "Resolved_By":          "e.resolved_by_user_id",
  "Latest_Activity        ": "'' AS Latest_Activity",
  "Score":                "r.score",
  "Input":                "r.input",
  "Labels":               "l.labels AS Labels",
}

common_fields_clean = {
  "Id":                   "STRING",
  "Process_Code":         "STRING",
  "Run_Id":               "INT64",
  "Roll_Up_Relationship": "STRING",
  "Workflow":             "STRING",
  "Status":               "STRING",
  "Group":                "STRING",
  "Assigned_To":          "STRING",
  "Ageing_Started_At":    "DATETIME",
  "Age_As_At_Export":     "FLOAT64",
  "Resolved_By":          "STRING",
  "Latest_Activity":      "STRING",
  "Score":                "FLOAT64",
  "Input":                "STRING",
  "Labels":               "STRING",
}

In [ ]:
last_common_fields = {
  "Fecha_Ejecucion":  "r.update_timestamp AS Fecha_Ejecucion",
}

last_common_clean = {
    "Fecha_Ejecucion":  "DATETIME",
}

In [ ]:


matched_fields_clean = {}
for k, v in matched_fields.items():
    new_v = v
    new_v = new_v.replace("r.", "")
    matched_fields_clean[k] = new_v

left_fields_clean = {}
for k, v in left_fields.items():
    new_v = v
    new_v = new_v.replace("r.rl_", "")
    left_fields_clean[k] = new_v

right_fields_clean = {}
for k, v in right_fields.items():
    new_v = v
    new_v = new_v.replace("r.rr_", "")
    right_fields_clean[k] = new_v

calculated_fields_clean = {}
for k, v in calculated_fields.items():
    new_v = v
    new_v = new_v.replace("r.cf_", "")
    calculated_fields_clean[k] = new_v

rules = {
    "b_": "BOOLEAN",
    "d_": "DATE",
    "dt_": "DATETIME",
    "f_": "FLOAT64",
    "i_": "INT64",
    "s_": "STRING",
    "t_": "TIME",
}

typed_recon_fields = {}
for k, v in (matched_fields_clean|left_fields_clean|right_fields_clean|calculated_fields_clean).items():
    tipo = None
    for prefix, t in rules.items():
        if v.startswith(prefix):
            tipo = t
            break
    typed_recon_fields[k] = tipo if tipo else "UNKNOWN"

rename_fields = {}
for k, v in typed_recon_fields.items():
    if v == "FLOAT64":
        new_key = f"{k}"
    elif v == "INT64":
        new_key = f"{k}"
    else:
        new_key = k
    rename_fields[new_key] = v

params = ",\n\t".join(f"`{k}` {v}" for k, v in (common_fields_clean|rename_fields|last_common_clean).items())

third_query = f"""
  CREATE TABLE `{PROJECT}.{DATASET}.{TABLE_NAME}` (
    \t{params}
  )
"""

query = third_query
print(query)